# Obteniendo los datos estadisticos de todos los archivos

Se calculan todas las variables estadisticas y se almacenan en un dataframe

### Importando librerias

In [1]:
import pandas as pd
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
import math
import numpy as np
import matplotlib.pyplot as plt

### Importar lista con nombres de archivos

*Cambiar el nombre de la lista según el sentimiento a analizar

In [2]:
files = pd.read_csv("Neutral.csv") ### MODIFICAR EL NOMBRE

### Se crea un diccionario para almacenar las variables de cada archivo

In [3]:
data_instance = {'M1': [],
                 'M2': [],
                 'M3': [],
                 'M4': [],
                 'C2': [],
                 'C3': [],
                 'C4': [],
                 'Skewness': [],
                 'Kurtosis': [],
                 'Mean': [],
                 'Absolute_mean': [],
                 'Peak_value': [],
                 'SRV': [],
                 'RMS': [],
                 'Crest_factor': [],
                 'Shape_factor': []}

### Ciclo para leer todos los archivos de cada sentimiento

In [4]:
for a in range(int(files.count())):
    name_doc = files._get_value (a, 'Name')
    data = pd.read_csv (name_doc)


    ########################################################################################################################################
    ################# Fourier #################
    raw_data = np.ravel(data.to_numpy())
    n = len(raw_data)
    yf = np.fft.fft(raw_data)       #Get fourier transform
    yfts = np.fft.fftshift(yf)
    yftsa = 1.0/n * np.abs(yfts)        #Remove complex numbers

    # Convert numpy array to DataFrame
    data = pd.DataFrame (yftsa)
    ##########################################################################################################################################    


    ################# Get statistical features #################
    m1 = float (stats.moment (data, moment = 1)) #First moment 
    m2 = float (stats.moment (data, moment = 2)) #Second moment
    m3 = float (stats.moment (data, moment = 3))#Third moment
    m4 = float (stats.moment (data, moment = 4)) #Fourth moment
    sc = float (m2 - math.sqrt (pow (m1,2))) #Second cumulant
    tc = float (m3 - (3*m1*m2) - (2*pow (m1,3))) #Third cumulant
    fc = float (m4 + (3*m3*m1) - (3*pow (m2,2)) + (12*m2*pow (m1,2)) - (6*pow (m1,4))) #Fourth cumulant
    skew = float (stats.skew (data)) #Skewness
    kurt = float (stats.kurtosis (data)) #Kurtosis
    mean = float(data.mean()) #Mean
    am = abs(float(data.mean())) #Absolute mean
    pv = 0.5*(float(data.max()) - float(data.min())) #Peak value
    da = abs(data)
    dsa = da**2
    srv = float(pow((dsa.sum()/int(dsa.count())),2)) #Square root value
    d2 = data**2
    rms = math.sqrt (float(d2.sum())/int(d2.count())) #Root mean square
    cf = pv/rms #Crest factor
    sf = rms/am #Shape factor

    ################# Append features to a dictionary #################
    data_instance ['M1'].append(m1)
    data_instance ['M2'].append(m2)
    data_instance ['M3'].append(m3)
    data_instance ['M4'].append(m4)
    data_instance ['C2'].append(sc)
    data_instance ['C3'].append(tc)
    data_instance ['C4'].append(fc)
    data_instance ['Skewness'].append(skew)
    data_instance ['Kurtosis'].append(kurt)
    data_instance ['Mean'].append(mean)
    data_instance ['Absolute_mean'].append(am)
    data_instance ['Peak_value'].append(pv)
    data_instance ['SRV'].append(srv)
    data_instance ['RMS' ].append(rms)
    data_instance ['Crest_factor'].append(cf)
    data_instance ['Shape_factor'].append(sf)

    del data

### Se crea un dataframe con todos los datos estadisticos y se guarda en un .csv

In [5]:
################# Create a dataframe from dictionary #################
data_stats = pd.DataFrame(data=data_instance)
data_stats.head()

,M1,M2,M3,M4,C2,C3,C4,Skewness,Kurtosis,Mean,Absolute_mean,Peak_value,SRV,RMS,Crest_factor,Shape_factor
0,0.0,2.467140e-08,4.040701e-10,1.562129e-11,2.467140e-08,4.040701e-10,1.561946e-11,104.271574,25661.282167,0.000025,0.000025,0.020083,6.399788e-16,0.000159,126.267163,6.355014
1,0.0,2.032446e-08,3.974761e-10,1.555269e-11,2.032446e-08,3.974761e-10,1.555145e-11,137.177356,37647.205701,0.000021,0.000021,0.020062,4.320172e-16,0.000144,139.152270,6.717878
2,0.0,5.195538e-08,7.700961e-10,2.851756e-11,5.195538e-08,7.700961e-10,2.850946e-11,65.027818,10561.558782,0.000035,0.000035,0.020038,2.825537e-15,0.000231,86.910024,6.654440
3,0.0,2.624430e-08,4.255293e-10,1.614212e-11,2.624430e-08,4.255293e-10,1.614006e-11,100.086840,23433.387467,0.000024,0.000024,0.020011,7.180802e-16,0.000164,122.242212,6.962912
4,0.0,3.468863e-08,5.537045e-10,2.084483e-11,3.468863e-08,5.537045e-10,2.084122e-11,85.703265,17320.040449,0.000028,0.000028,0.020075,1.256987e-15,0.000188,106.613976,6.806030


In [6]:
#Normalizacion
scaler = MinMaxScaler()
data_stats = pd.DataFrame (scaler.fit_transform(data_stats),columns=['M1','M2','M3','M4','C2','C3','C4','Skewness','Kurtosis','Mean','Absolute_mean','Peak_value','SRV','RMS','Crest_factor','Shape_factor'])

data_stats.to_csv (r"..\stats\Normalizated_data\Stats_neutral_fourier.csv")
data_stats.head()

,M1,M2,M3,M4,C2,C3,C4,Skewness,Kurtosis,Mean,Absolute_mean,Peak_value,SRV,RMS,Crest_factor,Shape_factor
0,0.0,0.244509,0.501364,0.542017,0.244509,0.501364,0.542130,0.595421,0.452639,0.367713,0.367713,0.905200,0.060065,0.471409,0.701732,0.603153
1,0.0,0.201076,0.493182,0.539637,0.201076,0.493182,0.539769,0.799882,0.665108,0.309337,0.309337,0.904194,0.040545,0.422915,0.790917,0.712649
2,0.0,0.517122,0.955567,0.989484,0.517122,0.955567,0.989525,0.351579,0.184973,0.525123,0.525123,0.903072,0.265203,0.704393,0.429317,0.693506
3,0.0,0.260225,0.527993,0.560089,0.260225,0.527993,0.560199,0.569419,0.413146,0.342873,0.342873,0.901822,0.067396,0.486545,0.673872,0.786589
4,0.0,0.344598,0.687046,0.723260,0.344598,0.687046,0.723370,0.480047,0.304777,0.410876,0.410876,0.904801,0.117978,0.566683,0.565700,0.739249
